<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# [Reconnaissance de caractères manuscrits](https://github.com/wikistat/Ateliers-Big-Data/2-MNIST) ([MNIST](http://yann.lecun.com/exdb/mnist/)) par apprentissage épais (*deep learning*) avec <a href="https://www.tensorflow.org/"><img src="https://avatars0.githubusercontent.com/u/15658638?s=200&v=4" style="max-width: 30px; display: inline" alt="TensorFlow"/></a> tensorflow et <a href="https://keras.io/"><img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" style="max-width: 100px; display: inline" alt="Keras"/></a>

### Résumé

## 1  Introduction

### 1.1 Objectif
Ce calepin reprend le même objectif que les calepins de l'[Atelier MNIST](https://github.com/wikistat/Ateliers-Big-Data/tree/master/2-MNIST) et sur les mêmes données mais en utilisant cette fois les librairies `Keras` et `TensorFlow` pour aborder l'apprentissage profond. Il est une adpatation du tutoriel de Keras.

### 1.2 Importation des librairies

In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

import pandas as pd
import numpy as np
import time

import keras.utils as ku
import keras.models as km
import keras.layers as kl
import keras.optimizers as ko

from sklearn.metrics import confusion_matrix

### 1.3 Lecture des données d'apprentissage et de test

Les données peuvent être préalablement téléchargées ou directement lues. Ce sont celles originales du site [MNIST DataBase](http://yann.lecun.com/exdb/mnist/) mais préalablement converties au format .csv, certes plus volumineux mais plus facile à lire. Attention le fichier `mnist_train.zip` présent dans le dépôt est compressé. 

In [3]:
# Lecture des données d'apprentissage
N_classes = 10

# path="" # Si les données sont dans le répertoire courant sinon:
path="/Users/bguillouet/Insa/TP_Insa/data/"
Dtrain=pd.read_csv(path+"mnist_train.csv",header=None)
X_train = Dtrain.values[:,:-1]
Y_train = Dtrain.values[:,-1]

Dtest=pd.read_csv(path+"mnist_test.csv",header=None)
X_test = Dtest.values[:,:-1]
Y_test = Dtest.values[:,-1]


*Attention*, avec Keras, la variable réponse doit être une matrice binaire où chaque classe est représentée par une indicatrice: pour chaque individu, l'élément de la colone correspondant à la classe à laquelle il appartient est à 1, sinon il est à 0. 

Keras possède une fonction `to_catergorical` permettant de convertir directement le vecteur variable `Y_train` de réponse en matrice (`array numpy`) indicatrice`Y_train_cat`.

In [4]:
Y_train_cat = ku.to_categorical(Y_train, N_classes)
Y_test_cat = ku.to_categorical(Y_test, N_classes)

## 2 Apprentissage et prévision du test

### 2.1 Perceptron
Première tentative d'appliquer un réseaux de neurone de type Perceptron classique avec 4 couches: 
* Dense: 52 neurones + Foncton d'activation *relu*
* *Dropout*: 20% des neurones tiré aléatoirement sont desactivés
* Dense: 52 neurones + Foncton d'activation *relu*
* *Dropout*: 20% des neurones tiré aléatoirement sont desactivés

Une dernière couche *softmax* fournit la classification

#### Apprentissage

In [5]:
# Paramètres
batch_size = 128
epochs = 20
# Définition du réseau
model = km.Sequential()
model.add(kl.Dense(512, activation='relu', input_shape=(784,)))
model.add(kl.Dropout(0.2))
model.add(kl.Dense(512, activation='relu'))
model.add(kl.Dropout(0.2))
model.add(kl.Dense(N_classes, activation='softmax'))
# Réumé
model.summary()
# apprentissage
model.compile(loss='categorical_crossentropy',
              optimizer=ko.RMSprop(),
              metrics=['accuracy'])
ts = time.time()
history = model.fit(X_train, Y_train_cat,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, Y_test_cat))
te = time.time()
t_train_mpl = te-ts

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 6s - loss: 14.5255 - acc: 0.

#### Résultats

Les résultats sont assez médiocres puisque l'on obtient seulement 20% d'images bien classées. 

In [6]:
score_mpl = model.evaluate(X_test, Y_test_cat, verbose=0)
predict_mpl = model.predict(X_test)
print('Test loss:', score_mpl[0])
print('Test accuracy:', score_mpl[1])
print("Time Running: %.2f seconds" %t_train_mpl )
pd.DataFrame(confusion_matrix(Y_test, predict_mpl.argmax(1)))

Test loss: 9.41022881012
Test accuracy: 0.416
Time Running: 156.66 seconds


,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,6,811,137,0,25,1
1,0,0,0,0,0,100,182,0,836,17
2,0,0,0,0,55,73,424,0,458,22
3,0,0,0,0,5,778,35,0,166,26
4,0,0,0,0,953,1,13,0,2,13
5,0,0,0,0,25,836,20,0,4,7
6,0,0,0,0,64,18,874,0,2,0
7,0,0,0,0,108,42,7,0,89,782
8,0,0,0,0,26,107,31,0,771,39
9,0,0,0,0,231,36,5,0,11,726


### 2.2 Effet de la normalisation des données

Le même réseau est utilisé mais avec normalisation préalable des données en divisant par leur valeur maximale (ici 255).

#### Apprentissage

In [7]:
batch_size = 128
epochs = 20
# Normalisation des données
X_train_norm = X_train/255
X_test_norm = X_test/255

model = km.Sequential()
model.add(kl.Dense(512, activation='relu', input_shape=(784,)))
model.add(kl.Dropout(0.2))
model.add(kl.Dense(512, activation='relu'))
model.add(kl.Dropout(0.2))
model.add(kl.Dense(N_classes, activation='softmax'))
model.summary()
# Apprentissage
model.compile(loss='categorical_crossentropy',
              optimizer=ko.RMSprop(),
              metrics=['accuracy'])
ts=time.time()
history = model.fit(X_train_norm, Y_train_cat,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test_norm, Y_test_cat))
te=time.time()
t_train_mpl_norm = te-ts

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 6s - loss: 0.2467 - acc: 0.9

#### Résultats

Le résultat est cette fois de 98% d'images bien classées!

In [8]:
score_mpl_norm = model.evaluate(X_test, Y_test_cat, verbose=0)
predict_mpl_norm = model.predict(X_test)
print('Test loss:', score_mpl_norm[0])
print('Test accuracy:', score_mpl_norm[1])
print("Time Running: %.2f seconds" %t_train_mpl_norm )
pd.DataFrame(confusion_matrix(Y_test, predict_mpl_norm.argmax(1)))

Test loss: 0.278843138504
Test accuracy: 0.9827
Time Running: 129.35 seconds


,0,1,2,3,4,5,6,7,8,9
0,971,1,1,0,1,1,4,0,0,1
1,1,1128,2,0,0,0,3,0,1,0
2,3,0,1015,1,1,0,5,4,2,1
3,0,0,3,988,0,4,0,6,3,6
4,1,0,1,0,964,0,6,3,0,7
5,2,0,0,3,0,877,6,0,3,1
6,2,2,0,0,2,3,949,0,0,0
7,2,4,8,0,0,0,0,1006,2,6
8,14,0,3,1,2,4,6,2,938,4
9,0,3,1,1,6,0,1,4,2,991


### 2.3 *Convolutional Network (ConvNet)*
Les propriété d'invariance par translation introduites par les couches opérant une convolution des images ont un impact important sur la qualité des résultats.
#### Réseau
Test d'un réseau de convolution constitué de 7 couches: 

* Une couche de convolution 2D, avec fenêtre de taille 3x3 et une fonction d'activation *relu*
* Une couche de convolution 2D, avec fenêtre de taille 3x3 et une fonction d'activation *relu*
* Une couche max pooling de fenêtre 2x2
* Une couche *dropout* où 25% des neurones sont desactivés
* Une couche *Flatten* transforme les images $N \times N$ en vecteurs $N^2$.
* Une couche classique de 128 neurones
* Une couche dropout ou 50% des neurones sont desactivés

Une couche *softmax* fournit la classification

#### Format des données

Dans les exemples précédents. Les données était "applaties". Une imade de $28\times 28=784$ pixels est considérée comme un vecteur. 

Pour pouvoir utiliser le principe de la convolution la structure des images est conservée. Une image n'est pas un vecteur de tailles $784\times 1$ mais une matrice de taille $28\times 28$. Ainsi `X_train` est réorganisée en cube ou multitableau de dimensions $60000\times 28\times 28$ pour être utilisé dans un réseau de convolution.

Avec **Keras** `X_train` doit même être de dimensions $60000\times 28\times 28\times 1$. La dernière dimension, de taille 1 peut paraitre inutile car dans le cas des données *MNIST* les pixels ne sont décrits qu'avec un seul niveau de gris. Cependant, des images couleurs en RGB sont généralement codées avec trois niveaux d'intensité (Rouge, Vert et Bleus) correspondant à la quatrième dimension comportant 3 valeurs. 

Noter également que l'utilisation des couches de convolution rend inutile la normalisation préalable des données.

In [ ]:
X_train_conv = X_train.reshape(60000, 28, 28, 1)
X_test_conv = X_test.reshape(10000, 28, 28, 1)

#### Apprentissage

In [ ]:
# descrition du réseau
model = km.Sequential()
model.add(kl.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28, 1), data_format="channels_last"))
model.add(kl.Conv2D(64, (3, 3), activation='relu'))
model.add(kl.MaxPooling2D(pool_size=(2, 2)))
model.add(kl.Dropout(0.25))
model.add(kl.Flatten())
model.add(kl.Dense(128, activation='relu'))
model.add(kl.Dropout(0.5))
model.add(kl.Dense(N_classes, activation='softmax'))
# Résumé
model.summary()
# Apprentissage
model.compile(loss="categorical_crossentropy",
              optimizer=ko.Adadelta(),
              metrics=['accuracy'])
ts=time.time()
model.fit(X_train_conv, Y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_conv, Y_test_cat))
te=time.time()
t_train_conv = te-ts

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________

#### Résultats

In [ ]:
score_conv = model.evaluate(X_test_conv, Y_test_cat, verbose=0)
predict_conv = model.predict(X_test_conv)
print('Test loss:', score_conv[0])
print('Test accuracy:', score_conv[1])
print("Time Running: %.2f seconds" %t_train_conv )
pd.DataFrame(confusion_matrix(Y_test, predict_conv.argmax(1)))

**Q** Commenter les résultats. Comparer avec les autres techniques d'apprentissage.

**Q** Comment améliorer encore ces résultats?